In [1]:
# Exercício de treinamento, criacao de um modelo reconhecedor dos digitos 3 e 7

#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *
from fastai.vision.all import *

matplotlib.rc('image', cmap='Greys') # para mostrar tudo cinza os vetores

     |████████████████████████████████| 719 kB 7.8 MB/s 
     |████████████████████████████████| 362 kB 65.5 MB/s 
     |████████████████████████████████| 1.2 MB 42.3 MB/s 
     |████████████████████████████████| 4.4 MB 52.4 MB/s 
     |████████████████████████████████| 1.1 MB 51.0 MB/s 
     |████████████████████████████████| 101 kB 13.3 MB/s 
     |████████████████████████████████| 212 kB 26.8 MB/s 
     |████████████████████████████████| 140 kB 30.9 MB/s 
     |████████████████████████████████| 596 kB 46.3 MB/s 
     |████████████████████████████████| 127 kB 49.3 MB/s 
     |████████████████████████████████| 271 kB 46.2 MB/s 
     |████████████████████████████████| 94 kB 2.1 MB/s 
     |████████████████████████████████| 144 kB 45.1 MB/s 
     |████████████████████████████████| 6.6 MB 42.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10

In [2]:
path = untar_data(URLs.MNIST_SAMPLE) # esse sample do reconhecedor universal só tem 3s e 7s
Path.BASE_PATH = path
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()
# podemos criar um Pytorch tensor pra representar a img, é um array do numpy injetavel na GPU
# imgTensor = tensor(Image.open(threes[0]))

In [3]:
# Criar listas para guardar todos os setes e os tres
sevenTensors = [tensor(Image.open(i)) for i in sevens]
threeTensors = [tensor(Image.open(i)) for i in threes]

# como nos temos listas, não é util para fazer computacoes matematicas
# um tensor pode ter vários "eixos/dimensões"
# podemos pegar a lista de imgs e "stackar" uma em cima da outra
# criando uma espécie de cubo 3D, com o qual dá pra fazer os calculos
stackedSevens = torch.stack(sevenTensors).float() / 255 # divide por 255 pq imagem tem 255 pixels
stackedThrees = torch.stack(threeTensors).float() / 255 # mas quando sao img como um float, valores de pixels sao entre 0 e 1
# NOMENCLATURA IMPORTANTE #
# rank = dimensoes de um tensor #
# shape = tamanho de cada dimensao #

In [19]:
# Precisamos medir proximidade de pixels (distância), para comparar os números com a média
# L2 Form: pega a MEAN do quadrado das diferencas e dps tira a raiz de tudo

a = stackedThrees[0] # vamos ver a distância desse 3 para a média do 3 e pro 7

distThree = ((a - stackedThrees.mean(0))**2).mean().sqrt()
distSeven = ((a - stackedSevens.mean(0))**2).mean().sqrt()

# e agora baseado na distância, dizer se o 3 escolhido é um 3 ou um 7
if (distThree < distSeven):
  print('é um 3')
else: 
  print('é um 7')

# Essa já é uma BASELINE válida para o modelo, é bem simples mas é sólida

tensor(0.1912)
tensor(0.2780)


In [26]:
# pegar a validation set
threeValidImg = (path/'valid'/'3').ls().sorted()
sevenValidImg = (path/'valid'/'7').ls().sorted()
threeValidationTensors = [tensor(Image.open(i)) for i in threeValidImg]
sevenValidationTensors = [tensor(Image.open(i)) for i in sevenValidImg]

# stackar tb
three_stackedValidation = torch.stack(threeValidationTensors).float() / 255
seven_stackedValidation = torch.stack(sevenValidationTensors).float() / 255

# len(threeValidImg), len(threes) # validation set / training set
# three_stackedValidation.shape() # bom habito checar a shape dos tensores

def mnsit_dist(a, b):
  return ((a-b)**2).mean((-1,-2)).sqrt() ## eixos (-1, -2) é importante ser definido
  #eixos -1 e -2 são os horizontal e vertical da img

# mnsit_dist(three_stackedValidation, stackedThrees.mean(0))
# essa chamada nao reclamaria de diferenca em shape, retornaria o calculo da dist
# para todos dentro do three_stackedValidation em relacao ao 3 ideal
# nome disso é BROADCASTING, o pytorch altera o rank de um tensor para um cálculo com tensor de rank diferente

(torch.Size([28, 28]), torch.Size([1010, 28, 28]))

In [33]:
# fc p/ definir se é um 3 ou 7
def isThree(a):
  return mnsit_dist(a, stackedThrees.mean(0)) < mnsit_dist(a, stackedSevens.mean(0))

# obs: ao converter bool return pra float, é 1.0 p true e 0.0 p false
# vamos calcular a accuracy, a precisão
acc_threes = isThree(three_stackedValidation).float().mean()
acc_sevens = 1 - (isThree(seven_stackedValidation).float().mean())
acc_threes, acc_sevens

(tensor(0.9584), tensor(0.9737))

In [ ]:
# (tensor(0.9584), tensor(0.9737)) FOI O OUTPUT
# ou seja, precisão de 95% para os 3 e 97% para os 7
# bom resultado! Mas são só 2 numeros, e sao bem diferentes, entao precisa melhorar muito mais!